In [3]:
import splitfolders as sf
from pathlib import Path

In [4]:
dataset_dir=Path("A:\AI DB\LSPD\Videos\photos_from_videos")

output_dir=Path("A:\AI DB\LSPD\Videos\photos_from_videos_pret")
output_dir.mkdir(exist_ok=True)

In [5]:
# split dataset 90% training 5% validation 5% testing
sf.ratio(dataset_dir,output_dir,seed=1337,ratio=(.9,.05,.05),group_prefix=None)

Copying files: 973644 files [57:40, 281.36 files/s]


In [78]:
import tensorflow as tf
import os
import splitfolders as sf
from pathlib import Path
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from sklearn.metrics import roc_curve, roc_auc_score
import math
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras import layers, models
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, concatenate, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB4
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.applications.efficientnet import preprocess_input
class CustomLearningRateScheduler(LearningRateScheduler):
    def __init__(self, schedule, patience=0, factor=0.1, min_lr=0, filepath=None, save_best_only=False, **kwargs):
        self.patience = patience
        self.factor = factor
        self.min_lr = min_lr
        self.filepath = filepath
        self.save_best_only = save_best_only
        super(CustomLearningRateScheduler, self).__init__(schedule, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        # Your custom logic here
        # You can access self.patience, self.factor, etc. here
        pass

In [79]:
def preprocess_image(image):
    try:
        # Perform preprocessing operations on the image
        # Example: resize, normalize, etc.
        # Make sure to return the preprocessed image
        preprocessed_image = preprocess_input(image)
        return preprocessed_image
    except Exception as e:
        # Handle the exception (e.g., log error, skip image)
        # For now, just return None to indicate that the image should be skipped
        print(f"Error processing image: {e}")
        return None

In [80]:
def create_datagen():
    return ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
    )

def create_generators(train_dir, validation_dir, test_dir, img_height, img_width, batch_size):
    train_datagen = create_datagen()
    validation_datagen = ImageDataGenerator(rescale=1./255,
        preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(rescale=1./255,
        preprocessing_function=preprocess_input
        )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle=False,
        class_mode='binary'
    )
    
    return train_generator, validation_generator, test_generator

In [81]:
def build_model():
    base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

    # Freeze the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)  # Adding Dropout for regularization
    predictions = Dense(1, activation='sigmoid', dtype='float32')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # Count trainable and non-trainable parameters
    trainable_count = np.sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    non_trainable_count = np.sum([tf.keras.backend.count_params(w) for w in model.non_trainable_weights])
    print("Trainable parameters:", trainable_count)
    print("Non-trainable parameters:", non_trainable_count)
    
    return model

In [82]:
def custom_schedule(epoch, lr):
    if epoch < 4:
        return lr
    else:
        return lr * 0.5

def create_callbacks():
    lr_scheduler = CustomLearningRateScheduler(
        schedule=custom_schedule,  # Use your custom schedule function here
        patience=2, 
        factor=0.5, 
        min_lr=1e-6, 
        filepath=r'A:\AI DB\LSPD\models\efficienttrainedonimagefromvideos.keras', 
        save_best_only=True
    )
    early_stopping = EarlyStopping(
        monitor='val_accuracy', 
        patience=3, 
        verbose=1, 
        mode='max'
    )
    return [lr_scheduler, early_stopping]


In [83]:
def train(train_dir, validation_dir, test_dir, img_height, img_width, batch_size):
    # Assuming that the dataset is organized as per Keras ImageDataGenerator requirements
    train_generator, validation_generator, test_generator = create_generators(
        train_dir, validation_dir, test_dir, img_height, img_width, batch_size)
    
    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes),y= train_generator.classes)
    class_weights = dict(enumerate(class_weights))

    model = build_model()
    callbacks = create_callbacks()

    # Train the model
    history = model.fit(
        train_generator,
        epochs=4,
        validation_data=validation_generator,
        class_weight=class_weights,
        callbacks=callbacks
    )

    # Unfreeze the last block of layers after a certain number of epochs (e.g., 5)
    for layer in model.layers[-20:]:
        layer.trainable = True

    # Compile the model after unfreezing
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Continue training with the unfrozen layers
    history_unfrozen = model.fit(
        train_generator,
        epochs=6,  # Continue training for additional epochs
        initial_epoch=3,  # Start from epoch 5 after unfreezing
        validation_data=validation_generator,
        class_weight=class_weights,
        callbacks=callbacks
    )
    
    # Concatenate the histories
    history.history['loss'].extend(history_unfrozen.history['loss'])
    history.history['accuracy'].extend(history_unfrozen.history['accuracy'])
    history.history['val_loss'].extend(history_unfrozen.history['val_loss'])
    history.history['val_accuracy'].extend(history_unfrozen.history['val_accuracy'])
        
    return history


In [84]:
def evaluate_model(test_generator, model,threshold):
    # Predict on the test data
    ypred = model.predict(test_generator)
    y_pred_classes = (ypred > threshold).astype(int)

    # Get actual classes
    y_true_classes = test_generator.classes

    # Create a confusion matrix
    cm = confusion_matrix(y_true_classes, y_pred_classes)

    # Normalize the confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Generate a classification report
    cr = classification_report(y_true_classes, y_pred_classes, digits=4)
    print("Classification Report:\n", cr)

    # Plot the confusion matrix with annotations positioned at the bottom
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, cmap='YlGnBu', fmt='d', xticklabels=['Normal', 'Porn'], yticklabels=['Normal', 'Porn'], annot_kws={'size': 16})

    # Add custom annotations at the bottom of each cell
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j + 0.5, i + 0.2, '{:.2f}%'.format(cm_normalized[i, j] * 100), ha='center', va='bottom', color='black', fontsize=14)

    plt.ylabel('Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=12)
    plt.title('Confusion Matrix', fontsize=16)
    plt.show()

    # Calculate accuracy score
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    print("Accuracy Score:", format(accuracy, '.4f'))

    # Calculate F1 score
    f1 = f1_score(y_true_classes, y_pred_classes)
    print("F1 Score:", format(f1, '.4f'))

In [85]:
def tune_threshold(test_generator, model):
    ypred = model.predict(test_generator)
    thresholds = np.arange(0.0, 1.0, 0.01)
    best_f1 = 0
    best_threshold = 0

    for threshold in thresholds:
        y_pred_classes = (ypred > threshold).astype(int)
        f1 = f1_score(test_generator.classes, y_pred_classes)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    print(f"Best threshold: {best_threshold} with F1 score: {best_f1}")
    return best_threshold

In [86]:
train_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\train'
validation_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\val'
test_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\test'

save_path=r'A:\AI DB\LSPD\models\inceptionTrainedOnVideos.keras'
# Define image dimensions and batch size
img_height, img_width = 380, 380
batch_size = 64

In [87]:
print("Training starts : ")
# Train the model using cross-validation
histories = train(train_dir, validation_dir, test_dir, img_height, img_width, batch_size)
print("Training finished .")
# Load the best model
model = tf.keras.models.load_model(save_path)


# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle=False,
        class_mode='binary'
    )
print("Evaluation : ")
evaluate_model(test_generator, model,0.5)
print("Calculating best threshold : ")
# Tune threshold
best_threshold = tune_threshold(test_generator, model)
print("Best threshold : ",str(best_threshold))
# Re-evaluate using the best threshold
print("Evaluation with new threshold : ")
evaluate_model(test_generator, model,best_threshold)

Training starts : 
Found 430877 images belonging to 2 classes.
Found 23937 images belonging to 2 classes.
Found 23939 images belonging to 2 classes.
Trainable parameters: 1837057
Non-trainable parameters: 17673823.0
Epoch 1/4
6733/6733 [==============================] - 11265s 2s/step - loss: 0.6897 - accuracy: 0.5554 - val_loss: 0.6758 - val_accuracy: 0.6236
Epoch 2/4
6733/6733 [==============================] - 11132s 2s/step - loss: 0.6850 - accuracy: 0.5816 - val_loss: 0.6813 - val_accuracy: 0.5934
Epoch 3/4
5119/6733 [=====================>........] - ETA: 42:43 - loss: 0.6836 - accuracy: 0.5882

KeyboardInterrupt: 